In [18]:
#! pip install -r requirements.txt --quiet

## Azure AI Search tool

The Azure AI Search tool in Agent Service connects an agent to a new or existing search index. You can use this tool to retrieve and summarize your indexed documents, grounding the agent's responses in your proprietary content.

🔗 [How to use an existing index with the Azure AI Search tool](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools-classic/azure-ai-search-samples?view=foundry-classic&pivots=python)

### Create an Azure AI Client
First, create an Azure AI Client using the endpoint of your Foundry project.

In [19]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


load_dotenv(override=True)

# Retrieve the endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False)
)

### Configure the Azure AI Search tool
Using the connection ID of your Azure AI Search service, configure the Azure AI Search tool to use your search index.

In [20]:
from azure.ai.agents.models import AzureAISearchTool, AzureAISearchQueryType
from azure.ai.projects.models import ConnectionType

# Define the Azure AI Search connection ID and index name
azure_ai_conn_id = project_client.connections.get_default(ConnectionType.AZURE_AI_SEARCH).id

# Find the index name on the Search Management > Indexes page of your Azure AI Search service
index_name = os.environ["SEARCH_INDEX"]

# Initialize the Azure AI Search tool
ai_search = AzureAISearchTool(
    index_connection_id=azure_ai_conn_id,
    index_name=index_name,
    query_type=AzureAISearchQueryType.VECTOR_SIMPLE_HYBRID,  # Use SIMPLE query type
    top_k=3,  # Retrieve the top 3 results
    filter="",  # Optional filter for search results
)

### Create an agent with the Azure AI Search tool enabled
Change the model to the one deployed in your project. You can find the model name on the Models tab of the Foundry portal. You can also change the agent's name and instructions to suit your needs.

In [21]:
instructions = """
You are a logistics assistant that answers questions about trucking loads,
shipments, and cargo operations.

Base all answers strictly on the information that is available to you.
If the available information is insufficient to answer a question,
clearly state that the information is not available.

Guidelines:
- Use only the information provided to you when answering.
- Do NOT invent shipment details, timelines, causes, or outcomes.
- Do NOT assume facts that are not explicitly stated.
- If multiple sources of information apply, combine them into a clear,
  factual response.
- If a question cannot be answered with the available information,
  say so explicitly.

Scope:
- Loads and shipment details
- Delivery deadlines and priorities
- Customer communications related to shipments
- Driver notes and operational narratives
- Contractual or SLA language that is explicitly provided

Out of scope:
- Real-time vehicle location
- Dispatch decisions or optimization
- Root-cause analysis beyond documented information
- Any data not present in the available information

Tone:
- Clear and professional
- Factual and concise
- No speculation
"""

# Define the model deployment name
model_deployment_name = os.environ["MODEL_DEPLOYMENT_NAME"]

# Create an agent with the Azure AI Search tool
agent = project_client.agents.create_agent(
    model=model_deployment_name,
    name="LogisticsInsightsAgent",
    instructions=instructions,
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_lxojCQRyIIjBfzXUDdq5MnG0


### Ask the agent questions about data in the index
Now that the agent is created, you can ask it questions about the data in your search index.

In [22]:
from azure.ai.agents.models import MessageRole, ListSortOrder

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Send a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="Using the available shipment documentation, list any trucking loads that show a departure origin in Arizona.",
)
print(f"Created message, ID: {message['id']}")

# Create and process a run with the specified thread and agent
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

# Check if the run failed
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages in the thread
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")

Created thread, ID: thread_IZpOUGZFKBhdwoXQ0HckMOUu
Created message, ID: msg_DdTaMdtlwrtahgoHIvtkyNDA
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.USER, Content: [{'type': 'text', 'text': {'value': 'Using the available shipment documentation, list any trucking loads that show a departure origin in Arizona.', 'annotations': []}}]
Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'The trucking loads with a departure origin in Arizona are:\n\n1. **Load L-7047**\n   - Origin: Phoenix, AZ\n   - Destination: Los Angeles, CA\n   - Delivery Deadline: 2025-04-13\n   - Priority: Low\n   - Weight: 42,924 lbs【3:0†source】\n\n2. **Load L-7185**\n   - Origin: Phoenix, AZ\n   - Destination: Chicago, IL\n   - Delivery Deadline: 2025-07-08\n   - Priority: Normal\n   - Weight: 25,321 lbs【3:1†source】\n\n3. **Load L-7015**\n   - Origin: Phoenix, AZ\n   - Destination: Chicago, IL\n   - Delivery Deadline: 2025-09-23\n   - Priority: Normal\n   - Weight: 6,624 lbs【3:2†sou

### Clean up resources
After you complete these operations, delete the agent to clean up resources.

In [ ]:
# Delete the agent
project_client.agents.delete_agent(agent.id)
print("Deleted agent")